<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석, 데이터 과학, 풀스택(백엔드, 프론트엔드) 개발 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>

## flask_login 라이브러리

## 1. flask_login 란?

- 사용자를 체크하는 기능을 담당하는 라이브러리
- 예:
  - 사용자가 로그인시, flask_login 라이브러리를 사용하면, 사용자 관련 session 정보를 HTTP response 에 넣어서 보내주고,
  - 이를 기반으로 flask 서버에서 사용자를 구별할 수 있는 기능을 제공

### flask_login 주요 동작 방식

1. 사용자가 로그인하면, 로그인 정보를 User class 에서 객체로 가져오고, LoginManager() 에 추가하여 세션 생성
   - flask 서버가 리턴시에 해당 세션 정보를 웹페이지에 송부
2. current_user 객체에 해당 객체가 저장됨
   - 주요 attribute
     - current_user.id : 사용자 ID (unicode 로 된 값, python string 이라고 봐도 됨)
       - python3 는 기본적으로 unicode 를 사용
     - current_user.is_authenticated : 사용자가 로그인되었는지를 나타내는 값 (True or False)
     - 이외의 attribute 는 User class 를 정의하면서, 필요에 따라 추가하면 됨
3. 로그인 후 웹페이지로 flask 서버 접근시, 전달받은 세션 정보를 기반으로 접근
   - 세션 정보에서 id를 추출해서, LoginManager() 에서 다루는 id 일 경우, @login_required 로 데코레이터가 추가된 API 접근 허용
4. 사용자가 로그아웃시 LoginManager() 에서 해당 id 제거

### 문제는 User class 구현 예가 많지 않음
- 즉, 각자의 아키텍쳐 기반 하에 생성 필요
- User class 에는 flask_login 라이브러리의 UserMixin 클래스를 상속받아서 구현하는 것만 가이드

### flask_login User class 가 지원해야할 변수/함수들
- is_authenticated: This property should return True if the user is authenticated
- is_active: This property should return True if this is an active user (계정 중지된 사용자 확인)
- is_anonymous: This property should return True if this is an anonymous user
- get_id(): This method must return a unicode that uniquely identifies this user, and can be used to load the user from the user_loader callback

> UserMixin 클래스 상속시 관련 기본 변수/함수가 상속되고, 필요시에만 override 하면 됨

## 2. flask_login 코드 구현

### flask_login 초기 설정 코드
- 세션 generation 을 위해, flask 에서 secret key 를 정의해줘야 함
```python
app.secret_key = os.urandom(24)
```

- flask_login 라이브러리에서 세션 관리
  - 다음 코드에서 "strong" 옵션을 사용할 시 Session 보안 적용
```python
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.session_protection = "strong"
```

### 사용자 session 생성
- User 클래스를 기반으로 사용자 객체를 생성한 후, flask_login.login_user() 함수에 해당 객체를 넣어주면,
- 해당 사용자 기반 session 이 생성됨
- 해당 session 은 HTTP response 에 넣어져서 사용자 웹브라우저에 송부됨
  - HTTP Response 에 Set-Cookie 옵션에 해당 session 정보를 넣어서 송부하면,
  - 사용자 웹브라우저에서는 추후 해당 서버 주소로 HTTP request 송부시 해당 session 정보를 자동으로 넣어서 요청함
  - flask_login 은 HTTP request 에서 자동으로 session 정보를 가져와서, 사용자를 구분함
- session 생성시, 사용자 HTTP request 에 들어 있는 사용자 IP address 와 user agent 등을 함께 참조해서 session 을 생성하므로,
  - 다른 사용자가 session 탈취하더라도, 사용자 IP address 와 user agent 까지 동일하게 HTTP request 에 넣어서 요청해야 하고,
  - 이 경우 이에 대한 응답은 다른 컴퓨터의 사용자가 아닌, 해당 사용자 IP 로 전송되므로, session 보안에 유용함
  
```python
from flask_login import login_user

user = User.create(user_email, blog_id)
login_user(user)
```

### login_manager 관련 사전 선언 필요 함수

> 다음 두 함수는 사전 선언 필요

#### 로그인 후 최초 current_user 호출시 다음 코드 호출

- 사용자 정보를 flask_login 과 함께 사용되는 User class 를 통해 가져옴

```python
@login_manager.user_loader
def load_user(user_id):
    return User.get(user_id)
```

#### @login_required 데코레이터로 로그인 후 접근 가능한 페이지 protection 가능
> 로그인이 안된 채로 해당 페이지 접근시 @login_manager.unauthorized_handler 에 정의된 함수를 호출하므로, 필요시 구현 원하는 형태로 구현 가능 <br>
  
```python
@login_manager.unauthorized_handler
def unauthorized():
    return make_response(jsonify(success=False), 401)
```

### User class 구현
- User class 는 UserMixin class 를 상속해야 함
- 속성으로 사용자를 구분할 수 있는 id 를 반드시 가지고 있어야 함
- 이외 코드에 대해서는 구현을 통해 설명

```python
class User(UserMixin):

    def __init__(self, user_id, user_email, blog_id):
        self.id = user_id
        self.user_email = user_email
        self.blog_id = blog_id

    def get_id(self):
        return str(self.id)

    @staticmethod
    def get(user_id):
        mysql_db = conn_mysqldb()
        db_cursor = mysql_db.cursor()
        sql = "SELECT * FROM user_info WHERE USER_ID = '" + \
            str(user_id) + "'"
        db_cursor.execute(sql)
        user = db_cursor.fetchone()
        if not user:
            db_cursor.close()
            return None

        print(user)
        user = User(user_id=user[0], user_email=user[1], blog_id=user[2])
        db_cursor.close()
        return user

    @staticmethod
    def find(user_email):
        mysql_db = conn_mysqldb()
        db_cursor = mysql_db.cursor()
        sql = "SELECT * FROM user_info WHERE USER_EMAIL = '" + \
            str(user_email) + "'"
        db_cursor.execute(sql)
        user = db_cursor.fetchone()
        if not user:
            db_cursor.close()
            return None

        print(user)
        user = User(user_id=user[0], user_email=user[1], blog_id=user[2])
        db_cursor.close()
        return user

    @staticmethod
    def create(user_email, blog_id):
        user = User.find(user_email)
        if user == None:
            mysql_db = conn_mysqldb()
            db_cursor = mysql_db.cursor()
            sql = "INSERT INTO user_info (USER_EMAIL, BLOG_ID) VALUES ('%s', '%s')" % (
                str(user_email), str(blog_id))
            db_cursor.execute(sql)
            mysql_db.commit()
            return User.find(user_email)
        else:
            return user
```

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
본 강의만 잘 정리하면, 데이터 분석, 데이터 과학, 풀스택(백엔드, 프론트엔드) 개발 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b></div>